<a href="https://colab.research.google.com/github/soumik12345/enhance-me/blob/mirnet/notebooks/enhance_me_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/soumik12345/enhance-me -b mirnet
!pip install wandb streamlit

In [ ]:
import sys
sys.path.append("./enhance-me")

from PIL import Image
from enhance_me import commons
from enhance_me.mirnet import MIRNet

In [ ]:
#@title MIRNet Train Configs

experiment_name = 'lol_dataset_128' #@param {type:"string"}
image_size = 128 #@param {type:"integer"}
dataset_label = 'lol' #@param ["lol"]
apply_random_horizontal_flip = True #@param {type:"boolean"}
apply_random_vertical_flip = True #@param {type:"boolean"}
apply_random_rotation = True #@param {type:"boolean"}
wandb_api_key = '' #@param {type:"string"}
val_split = 0.1 #@param {type:"slider", min:0.1, max:1.0, step:0.1}
batch_size = 4 #@param {type:"integer"}
num_recursive_residual_groups = 3 #@param {type:"slider", min:1, max:5, step:1}
num_multi_scale_residual_blocks = 2 #@param {type:"slider", min:1, max:5, step:1}
learning_rate = 1e-4 #@param {type:"number"}
epsilon = 1e-3 #@param {type:"number"}
epochs = 50 #@param {type:"slider", min:10, max:100, step:5}

In [ ]:
mirnet = MIRNet(
    experiment_name=experiment_name,
    image_size=image_size,
    dataset_label=dataset_label,
    val_split=val_split,
    batch_size=batch_size,
    apply_random_horizontal_flip=apply_random_horizontal_flip,
    apply_random_vertical_flip=apply_random_vertical_flip,
    apply_random_rotation=apply_random_rotation,
    wandb_api_key=None if wandb_api_key == '' else wandb_api_key
)

In [ ]:
mirnet.build_model(
    num_recursive_residual_groups=num_recursive_residual_groups,
    num_multi_scale_residual_blocks=num_multi_scale_residual_blocks,
    learning_rate=learning_rate,
    epsilon=epsilon
)

In [ ]:
history = mirnet.train(epochs=epochs)

In [ ]:
for index, low_image_file in enumerate(mirnet.test_low_images):
    original_image = Image.open(low_image_file)
    enhanced_image = mirnet.infer(original_image)
    ground_truth = Image.open(mirnet.test_enhanced_images[index])
    commons.plot_results(
        [original_image, ground_truth, ground_truth],
        ["Original Image", "Ground Truth", "Enhanced Image"],
        (18, 18)
    )